## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-30-06-52-18 +0000,nypost,West Virginia University football game holds m...,https://nypost.com/2025/11/30/us-news/wvu-foot...
1,2025-11-30-06-22-36 +0000,bbc,Ukraine hits tankers in Black Sea in escalatio...,https://www.bbc.com/news/articles/cp9ky5z2gx0o...
2,2025-11-30-06-16-52 +0000,nyt,Cory Booker Weds Alexis Lewis in a Private Cer...,https://www.nytimes.com/2025/11/30/style/cory-...
3,2025-11-30-06-16-39 +0000,nypost,"Rep. Troy Nehls, a Trump ally, will not seek r...",https://nypost.com/2025/11/30/us-news/texas-re...
4,2025-11-30-05-57-27 +0000,bbc,Four killed in shooting at child's birthday pa...,https://www.bbc.com/news/articles/cvg81p61dnmo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2325/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,26
81,up,10
52,dead,8
22,russia,8
138,new,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
91,2025-11-29-19-34-03 +0000,nypost,White House hits back after Dem blames Trump f...,https://nypost.com/2025/11/29/us-news/white-ho...,55
96,2025-11-29-19-03-00 +0000,wsj,Why Russia and China Are Sitting Out Venezuela...,https://www.wsj.com/world/americas/why-russia-...,51
76,2025-11-29-20-49-01 +0000,cbc,Maduro slams 'colonial threat' as Trump says V...,https://www.cbc.ca/news/world/trump-maduro-ven...,50
84,2025-11-29-20-06-00 +0000,wsj,Trump Declares That Airspace Around Venezuela ...,https://www.wsj.com/politics/national-security...,50
3,2025-11-30-06-16-39 +0000,nypost,"Rep. Troy Nehls, a Trump ally, will not seek r...",https://nypost.com/2025/11/30/us-news/texas-re...,47


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
91,55,2025-11-29-19-34-03 +0000,nypost,White House hits back after Dem blames Trump f...,https://nypost.com/2025/11/29/us-news/white-ho...
19,38,2025-11-30-03-50-26 +0000,nypost,"New York, New Jersey could get up to 5 inches ...",https://nypost.com/2025/11/29/us-news/ny-nj-co...
129,35,2025-11-29-13-50-50 +0000,nypost,Russia attack on Kyiv kills three as efforts t...,https://nypost.com/2025/11/29/world-news/russi...
37,33,2025-11-30-01-08-54 +0000,nypost,"Heartless son allegedly beat his parents, set ...",https://nypost.com/2025/11/29/us-news/lunatic-...
86,29,2025-11-29-20-03-34 +0000,nypost,Pope Leo visits Turkey’s famous Blue Mosque in...,https://nypost.com/2025/11/29/world-news/pope-...
0,26,2025-11-30-06-52-18 +0000,nypost,West Virginia University football game holds m...,https://nypost.com/2025/11/30/us-news/wvu-foot...
102,26,2025-11-29-18-32-30 +0000,nypost,ICE protesters clash with cops at NYC governme...,https://nypost.com/2025/11/29/us-news/150-ice-...
76,24,2025-11-29-20-49-01 +0000,cbc,Maduro slams 'colonial threat' as Trump says V...,https://www.cbc.ca/news/world/trump-maduro-ven...
47,23,2025-11-29-23-33-57 +0000,nyt,"Tom Stoppard, Award-Winning Playwright of Witt...",https://www.nytimes.com/2025/11/29/theater/tom...
67,23,2025-11-29-21-45-09 +0000,nypost,Parents of slain Israeli hostage still held in...,https://nypost.com/2025/11/29/world-news/paren...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
